In [2]:
import json
import pandas as pd

In [20]:
with open('../noaa.json') as fi:
    credentials = json.load(fi)

In [17]:
stations_df = pd.read_csv('../datasets/weather_stations.csv')

In [21]:
credentials

{'token': 'vQAHfhnCqxFnzqGXVjDCRolQvblqEEGB'}

In [22]:
credentials['token']

'vQAHfhnCqxFnzqGXVjDCRolQvblqEEGB'

In [201]:
# selected from web-cdo noaa
stations = ['GHCND:US1TXDA0101',
'GHCND:USW00013960',
'GHCND:US1TXDA0053',
'GHCND:US1TXDA0076',
'GHCND:US1TXDA0048', # 2013-09-01 to 2024-05-23 Coverage 53% 
'GHCND:US1TXDA0112',
'GHCND:US1TXDA0013',
'GHCND:USW00003971']

In [213]:
# import requests
# import pandas as pd
# from geopy.distance import geodesic

# # NOAA API token
# api_token = credentials['token']  # Replace with your actual API token

# # Function to find stations close to a given lat/lon
# def find_nearby_stations():
#     stations_url = 'https://www.ncei.noaa.gov/cdo-web/api/v2/stations?locationid=FIPS:48113'
#     headers = {
#         'token': api_token
#     }
#     response = requests.get(stations_url, headers=headers)
#     if response.status_code == 200:
#         return response.json().get('results', [])
#     else:
#         print(f"Failed to get stations: {response.status_code}")
#         return []

# # Find nearby stations within a 50 km radius
# nearby_stations = find_nearby_stations()

In [202]:
start_date = '2022-04-01'
end_date = '2023-03-31'
# station_id = nearby_stations[4]['id']
station_id = stations[0]

In [214]:
import requests
import pandas as pd

In [ ]:
api_token = credentials['token']

In [203]:
data_url = 'https://www.ncei.noaa.gov/cdo-web/api/v2/data?'
params = {
    'datasetid': 'GHCND',
    'startdate': start_date,
    'enddate': end_date,
    'stationid': station_id,
    # 'datatypeid': ['TAVG', 'AWND'],  # Daily average temperature and average windspeed
    # 'units': 'metric',  # Use 'standard' for Fahrenheit
    # 'limit': 1000
}
headers = {
    'token': api_token
}
request_url = requests.Request('GET', data_url, params=params).prepare().url
print("Request URL for finding nearby stations:", request_url)
response = requests.get(data_url, headers=headers, params=params)
# Check the response status code and content
print("Response status code:", response.status_code)

# Print the raw response content for debugging
print("Response content:", response.text)

Request URL for finding nearby stations: https://www.ncei.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&startdate=2022-04-01&enddate=2023-03-31&stationid=GHCND%3AUS1TXDA0101
Response status code: 200
Response content: {"metadata":{"resultset":{"offset":1,"count":579,"limit":25}},"results":[{"date":"2022-04-01T00:00:00","datatype":"PRCP","station":"GHCND:US1TXDA0101","attributes":",,N,0600","value":0},{"date":"2022-04-01T00:00:00","datatype":"SNOW","station":"GHCND:US1TXDA0101","attributes":",,N,0600","value":0},{"date":"2022-04-02T00:00:00","datatype":"PRCP","station":"GHCND:US1TXDA0101","attributes":",,N,0600","value":20},{"date":"2022-04-03T00:00:00","datatype":"PRCP","station":"GHCND:US1TXDA0101","attributes":",,N,0600","value":0},{"date":"2022-04-03T00:00:00","datatype":"SNOW","station":"GHCND:US1TXDA0101","attributes":",,N,0600","value":0},{"date":"2022-04-04T00:00:00","datatype":"PRCP","station":"GHCND:US1TXDA0101","attributes":",,N,0600","value":0},{"date":"2022-04-04T00:00:00","

In [150]:
response

<Response [200]>

In [240]:
# response.json()

In [205]:
# Initialize an empty list to store the data from all stations
all_station_data = []

# Loop through each nearby station to get the data
for station in stations:
    # station_id = station['id']
    data_url = 'https://www.ncei.noaa.gov/cdo-web/api/v2/data?'
    params = {
        'datasetid': 'GHCND',
        'startdate': start_date,
        'enddate': end_date,
        'stationid': station,
        'units': 'metric',
        'limit': 1000
    }
    headers = {
        'token': api_token
    }
    request_url = requests.Request('GET', data_url, params=params).prepare().url
    print("Request URL for downloading weather data for station:", request_url)
    
    # Make the request
    response = requests.get(data_url, headers=headers, params=params)
    print("Response status code:", response.status_code)
    print("Response content:", response.text)  # Print the raw response content
    
    if response.status_code == 200:
        try:
            data = response.json().get('results', [])
            if not data:
                print("No data found for the specified station and date range.")
            else:
                # Append the data to the list
                all_station_data.extend(data)
        except ValueError as e:
            print("Error decoding JSON:", e)
            print("Response content:", response.text)
    else:
        print(f"Failed to get data for station {station_id}: {response.status_code}, {response.text}")
        
if all_station_data:
    df = pd.DataFrame(all_station_data)
    if not df.empty:
        # NOAA provides TAVG in tenths of degrees Celsius, so divide by 10
        df['value'] = df.apply(lambda row: row['value'] / 10 if row['datatype'] == 'TAVG' else row['value'], axis=1)
        daily_averages = df.groupby(['date', 'datatype'])['value'].mean().unstack()
        
        # Save to CSV
        daily_averages.to_csv('daily_averages.csv')
        
        print('Weather data download and processing complete!')
    else:
        print('No data available for the specified date range and location.')
else:
    print('No data was retrieved from any station.')

Request URL for downloading weather data for station: https://www.ncei.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&startdate=2022-04-01&enddate=2023-03-31&stationid=GHCND%3AUS1TXDA0101&units=metric&limit=1000
Response status code: 200
Response content: {"metadata":{"resultset":{"offset":1,"count":579,"limit":1000}},"results":[{"date":"2022-04-01T00:00:00","datatype":"PRCP","station":"GHCND:US1TXDA0101","attributes":",,N,0600","value":0.0},{"date":"2022-04-01T00:00:00","datatype":"SNOW","station":"GHCND:US1TXDA0101","attributes":",,N,0600","value":0.0},{"date":"2022-04-02T00:00:00","datatype":"PRCP","station":"GHCND:US1TXDA0101","attributes":",,N,0600","value":2.0},{"date":"2022-04-03T00:00:00","datatype":"PRCP","station":"GHCND:US1TXDA0101","attributes":",,N,0600","value":0.0},{"date":"2022-04-03T00:00:00","datatype":"SNOW","station":"GHCND:US1TXDA0101","attributes":",,N,0600","value":0.0},{"date":"2022-04-04T00:00:00","datatype":"PRCP","station":"GHCND:US1TXDA0101","attributes":",,N,

In [234]:
# daily_averages

In [230]:
df['datatype'].unique()

array(['PRCP', 'SNOW', 'DAPR', 'MDPR', 'ADPT', 'ASLP', 'ASTP', 'AWBT',
       'AWND', 'RHAV', 'RHMN', 'RHMX', 'SNWD', 'TAVG', 'TMAX', 'TMIN',
       'WDF2', 'WDF5', 'WSF2', 'WSF5', 'WT01', 'WT03', 'WT08', 'WESD',
       'WT02'], dtype=object)

In [239]:
df[df['datatype']=='TAVG']['station'].unique()

array(['GHCND:USW00013960'], dtype=object)

In [3]:
data = pd.read_csv('../datasets/Dallas_stations_data.csv')

In [235]:
# data

In [4]:
data.columns

Index(['STATION', 'NAME', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'DATE', 'AWND',
       'AWND_ATTRIBUTES', 'DAPR', 'DAPR_ATTRIBUTES', 'MDPR', 'MDPR_ATTRIBUTES',
       'PGTM', 'PGTM_ATTRIBUTES', 'PRCP', 'PRCP_ATTRIBUTES', 'SNOW',
       'SNOW_ATTRIBUTES', 'SNWD', 'SNWD_ATTRIBUTES', 'TAVG', 'TAVG_ATTRIBUTES',
       'TMAX', 'TMAX_ATTRIBUTES', 'TMIN', 'TMIN_ATTRIBUTES', 'WDF2',
       'WDF2_ATTRIBUTES', 'WDF5', 'WDF5_ATTRIBUTES', 'WESD', 'WESD_ATTRIBUTES',
       'WESF', 'WESF_ATTRIBUTES', 'WSF2', 'WSF2_ATTRIBUTES', 'WSF5',
       'WSF5_ATTRIBUTES', 'WT01', 'WT01_ATTRIBUTES', 'WT02', 'WT02_ATTRIBUTES',
       'WT03', 'WT03_ATTRIBUTES', 'WT04', 'WT04_ATTRIBUTES', 'WT05',
       'WT05_ATTRIBUTES', 'WT06', 'WT06_ATTRIBUTES', 'WT08',
       'WT08_ATTRIBUTES'],
      dtype='object')

In [237]:
# data['TAVG']

In [5]:
data['TAVG'].dropna().count()

732

In [13]:
data['WT02'].unique()

array([nan,  1.])

In [5]:
data.groupby('STATION')['TAVG'].value_counts().reset_index()['STATION'].unique()

array(['USW00013960'], dtype=object)

In [36]:
data.groupby('STATION')['WT04'].value_counts()

STATION      WT04
USW00003971  1.0     2
USW00013960  1.0     5
Name: count, dtype: int64

USW00003971

In [8]:
data.columns

Index(['STATION', 'NAME', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'DATE', 'AWND',
       'AWND_ATTRIBUTES', 'DAPR', 'DAPR_ATTRIBUTES', 'MDPR', 'MDPR_ATTRIBUTES',
       'PGTM', 'PGTM_ATTRIBUTES', 'PRCP', 'PRCP_ATTRIBUTES', 'SNOW',
       'SNOW_ATTRIBUTES', 'SNWD', 'SNWD_ATTRIBUTES', 'TAVG', 'TAVG_ATTRIBUTES',
       'TMAX', 'TMAX_ATTRIBUTES', 'TMIN', 'TMIN_ATTRIBUTES', 'WDF2',
       'WDF2_ATTRIBUTES', 'WDF5', 'WDF5_ATTRIBUTES', 'WESD', 'WESD_ATTRIBUTES',
       'WESF', 'WESF_ATTRIBUTES', 'WSF2', 'WSF2_ATTRIBUTES', 'WSF5',
       'WSF5_ATTRIBUTES', 'WT01', 'WT01_ATTRIBUTES', 'WT02', 'WT02_ATTRIBUTES',
       'WT03', 'WT03_ATTRIBUTES', 'WT04', 'WT04_ATTRIBUTES', 'WT05',
       'WT05_ATTRIBUTES', 'WT06', 'WT06_ATTRIBUTES', 'WT08',
       'WT08_ATTRIBUTES'],
      dtype='object')

In [6]:
data[data['STATION']=='USW00003971'][['STATION', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'DATE','WT01','WT02','WT03','WT04','WT05','WT06','WT08']]

,STATION,LATITUDE,LONGITUDE,ELEVATION,DATE,WT01,WT02,WT03,WT04,WT05,WT06,WT08
564,USW00003971,32.67816,-96.86771,200.4,2022-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
565,USW00003971,32.67816,-96.86771,200.4,2022-04-02,NaN,NaN,NaN,NaN,NaN,NaN,1.0
566,USW00003971,32.67816,-96.86771,200.4,2022-04-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN
567,USW00003971,32.67816,-96.86771,200.4,2022-04-04,1.0,NaN,1.0,NaN,NaN,NaN,NaN
568,USW00003971,32.67816,-96.86771,200.4,2022-04-05,1.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1291,USW00003971,32.67816,-96.86771,200.4,2024-03-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1292,USW00003971,32.67816,-96.86771,200.4,2024-03-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1293,USW00003971,32.67816,-96.86771,200.4,2024-03-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1294,USW00003971,32.67816,-96.86771,200.4,2024-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
data[data['STATION']=='USW00013960'][['STATION', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'DATE', 'AWND', 'DAPR', 'MDPR', 'PGTM', 'PRCP', 'SNOW', 'SNWD', 'TAVG','TMAX','TMIN', 'WDF2','WDF5', 'WESD', 'WESF','WSF2', 'WSF5', 'WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06','WT08']]

,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION,DATE,AWND,DAPR,MDPR,PGTM,...,WESF,WSF2,WSF5,WT01,WT02,WT03,WT04,WT05,WT06,WT08
3942,USW00013960,"DALLAS FAA AIRPORT, TX US",32.83839,-96.83583,147.6,2022-04-01,4.5,NaN,NaN,NaN,...,NaN,9.8,13.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3943,USW00013960,"DALLAS FAA AIRPORT, TX US",32.83839,-96.83583,147.6,2022-04-02,2.8,NaN,NaN,NaN,...,NaN,6.7,13.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3944,USW00013960,"DALLAS FAA AIRPORT, TX US",32.83839,-96.83583,147.6,2022-04-03,5.1,NaN,NaN,NaN,...,NaN,10.3,15.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3945,USW00013960,"DALLAS FAA AIRPORT, TX US",32.83839,-96.83583,147.6,2022-04-04,4.8,NaN,NaN,NaN,...,NaN,9.8,16.1,1.0,NaN,1.0,NaN,NaN,NaN,NaN
3946,USW00013960,"DALLAS FAA AIRPORT, TX US",32.83839,-96.83583,147.6,2022-04-05,3.7,NaN,NaN,NaN,...,NaN,10.7,15.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4669,USW00013960,"DALLAS FAA AIRPORT, TX US",32.83839,-96.83583,147.6,2024-03-28,3.0,NaN,NaN,NaN,...,NaN,7.2,10.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4670,USW00013960,"DALLAS FAA AIRPORT, TX US",32.83839,-96.83583,147.6,2024-03-29,6.8,NaN,NaN,NaN,...,NaN,12.5,19.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4671,USW00013960,"DALLAS FAA AIRPORT, TX US",32.83839,-96.83583,147.6,2024-03-30,5.5,NaN,NaN,NaN,...,NaN,8.9,13.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4672,USW00013960,"DALLAS FAA AIRPORT, TX US",32.83839,-96.83583,147.6,2024-03-31,4.8,NaN,NaN,NaN,...,NaN,10.3,17.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
